# 1. Librerías y dataset

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.impute import  SimpleImputer

In [2]:
# Dataframe
df = pd.read_csv("train.csv", sep=",")

# Conjunto de Test
test = pd.read_csv("test_x.csv", sep=",")

In [3]:
y = pd.DataFrame([])

In [4]:
y['subscripcion_deposito'] = df['Subscripcion Deposito']

In [5]:
y.rename(columns={'subscripcion_deposito': 'subscripcion_deposito'}, inplace=True)

In [6]:
# Eliminamos la columna ID
df.drop('Id', axis=1,  inplace=True)
test.drop('Id', axis=1,  inplace=True)

# Eliminamos la columna Y
df.drop('Subscripcion Deposito', axis=1,  inplace=True)

In [7]:
# df

In [8]:
# Renombramos columnas para reducir su extensión
nom_columns = { 'Edad': 'edad',
                'Trabajo': 'trabajo',
                'Estado Civil': 'estado_civil',
                'Educacion': 'educacion',
                'Credito por Defecto': 'default_credit',
                'Prestamo Vivienda': 'prest_vivienda',
                'Prestamo Personal': 'prest_personal',
                'Contacto': 'contact',
                'Mes': 'mes',
                'Dia Semana': 'dia_semana',
                'Duracion': 'duracion',
                'Cantidad Contactos Realizados': 'num_contact_realizados',
                'Cantidad Dias desde Ultimo Contacto': 'dias_ultimo_contacto',
                'Cantidad Contactos Previos Realizados': 'contactos_prev_realizados',
                'Resultado Campaña Anterior': 'resultado_camp_anterior',
                'Tasa Variacion Empleo': 'tasa_variacion_empleo',
                'Indice Precios Consumidor': 'ipc',
                'Indice Confianza Consumidor': 'ict',
                'Tasa Interes 3 meses': 'ti3m',
                'Cantidad Empleados': 'num_trabajadores'}

df.rename(columns=nom_columns, inplace=True)
test.rename(columns=nom_columns, inplace=True)

In [9]:
# Cambiamos los valores string "valores_que_faltan" por NaN
df = df.replace({'valor_que_falta': np.nan})
df['dias_ultimo_contacto'] = df['dias_ultimo_contacto'].replace({999: np.nan})

test = test.replace({'valor_que_falta': np.nan})
test['dias_ultimo_contacto'] = test['dias_ultimo_contacto'].replace({999: np.nan})

# 2. Funciones 

## 2.1. Contar valores nulos para todas las columnas del dataset

In [10]:
def conteo_valores_nulos( dataset: pd.DataFrame) -> pd.DataFrame:
    # revisamos valores nulos por columna
    missing_percentage = dataset.isnull().sum()
    missing_percentage = missing_percentage.reset_index()
    missing_percentage.columns = ['column_name', 'total_nan']
    missing_percentage['percentage_nan'] = missing_percentage['total_nan']/len(dataset)*100
    missing_percentage_sorted = missing_percentage.sort_values(by="percentage_nan", ascending=False)
    return missing_percentage_sorted

## 2.2. Contar valores únicos por columna, incluye valores null / nan

In [11]:
def conteo_valores_unicos(dataset: pd.DataFrame, column_name: str) -> pd.DataFrame:
    return dataset[column_name].value_counts(dropna=False)

## 2.3. Completar con la moda de una columna respecto de otra

In [12]:
def fill_with_mode(row: pd.Series, column_name: str, mode_per_grp_col: pd.DataFrame, group_col: str) -> pd.Series:
    """
    Imputa el valor NaN en una columna específica usando la moda del grupo al que pertenece cada fila.
    
    Parámetros:
    - row (pd.Series): Una fila del DataFrame.
    - column_name (str): El nombre de la columna donde imputar los NaN.
    - mode_per_grp_col (dict or pd.Series): Un diccionario o Serie con las modas para cada grupo.
    - group_col (str): El nombre de la columna que define los grupos.
    
    Retorna:
    - El valor imputado si es NaN, o el valor original si no lo es.
    """
    # Verificar si el valor en la columna especificada es NaN y si el grupo de trabajo tiene una moda definida
    if pd.isna(row[column_name]) and row[group_col] in mode_per_grp_col:
        return mode_per_grp_col[row[group_col]]
    return row[column_name]

# 3. Trabajando con la data

In [13]:
len(df)

32538

In [14]:
len(test)

12769

In [15]:
conteo_valores_nulos(df)

,column_name,total_nan,percentage_nan
12,dias_ultimo_contacto,31356,96.367324
4,default_credit,6766,20.794148
3,educacion,1389,4.268855
5,prest_vivienda,765,2.351097
6,prest_personal,765,2.351097
1,trabajo,264,0.811359
2,estado_civil,58,0.178253
0,edad,0,0.000000
14,resultado_camp_anterior,0,0.000000
18,ti3m,0,0.000000


In [16]:
conteo_valores_nulos(test)

,column_name,total_nan,percentage_nan
12,dias_ultimo_contacto,12276,96.139087
4,default_credit,2634,20.628084
3,educacion,509,3.986217
5,prest_vivienda,330,2.584384
6,prest_personal,330,2.584384
1,trabajo,105,0.822304
2,estado_civil,33,0.258438
0,edad,0,0.000000
14,resultado_camp_anterior,0,0.000000
18,ti3m,0,0.000000


## 3.1. Valores nulos por columna

## 3.2. Imputando datos a valores NaN

### 3.2.1. Imputando data con la moda global de columna

In [17]:
# Inicializa el SimpleImputer
imputer = SimpleImputer(strategy='most_frequent')

# Columnas que necesitan ser imputadas
columns_to_impute = ['prest_personal', 'prest_vivienda', 'default_credit', 'estado_civil']

# Imputa los valores faltantes en las columnas seleccionadas
df[columns_to_impute] = imputer.fit_transform(df[columns_to_impute])

In [18]:
# Inicializa el SimpleImputer
imputer = SimpleImputer(strategy='most_frequent')

# Columnas que necesitan ser imputadas
columns_to_impute = ['prest_personal', 'prest_vivienda', 'default_credit', 'estado_civil']

# Imputa los valores faltantes en las columnas seleccionadas
test[columns_to_impute] = imputer.fit_transform(test[columns_to_impute])

In [19]:
#imputer = SimpleImputer(strategy='most_frequent')
#df['prest_personal'] = imputer.fit_transform(df[['prest_personal']]).ravel()
#df['prest_vivienda'] = imputer.fit_transform(df[['prest_vivienda']]).ravel()
#df['default_credit'] = imputer.fit_transform(df[['default_credit']]).ravel()
#df['estado_civil'] = imputer.fit_transform(df[['estado_civil']]).ravel() 

In [20]:
conteo_valores_nulos(test)

,column_name,total_nan,percentage_nan
12,dias_ultimo_contacto,12276,96.139087
3,educacion,509,3.986217
1,trabajo,105,0.822304
0,edad,0,0.000000
11,num_contact_realizados,0,0.000000
18,ti3m,0,0.000000
17,ict,0,0.000000
16,ipc,0,0.000000
15,tasa_variacion_empleo,0,0.000000
14,resultado_camp_anterior,0,0.000000


In [21]:
#imputer = SimpleImputer(strategy='most_frequent')
#test['prest_personal'] = imputer.fit_transform(test[['prest_personal']]).ravel()
#test['prest_vivienda'] = imputer.fit_transform(test[['prest_vivienda']]).ravel()
#test['default_credit'] = imputer.fit_transform(test[['default_credit']]).ravel()
#test['estado_civil'] = imputer.fit_transform(test[['estado_civil']]).ravel()

In [22]:
# conteo_valores_nulos(df)

In [23]:
# Datos que no se podrán relacionar
#len(df[(df.educacion.isna()) & (df.trabajo.isna())])

In [24]:
#len(test[(test.educacion.isna()) & (test.trabajo.isna())])

### 3.2.2. Imputando data con la moda respecto de otra columna

#### 3.2.2.1. Completamos Educacion con la moda de sus grupos respectivos en Trabajo

In [25]:
mode_per_work = df.dropna(subset=['educacion']).groupby('trabajo')['educacion'].agg(lambda x: pd.Series.mode(x).iloc[0])
mode_per_work

trabajo
administración                 grado_universitario
administrador                  grado_universitario
desempleado                    secundaria_completa
empleada_domestica                primaria_4_grado
empleado_ por_cuenta_propia    grado_universitario
emprendedor                    grado_universitario
estudiante                     secundaria_completa
retirado                          primaria_4_grado
servicios                      secundaria_completa
tecnico                          curso_profesional
trabajador_industria              primaria_3_grado
Name: educacion, dtype: object

In [26]:
df['educacion'] = df.apply(fill_with_mode, args=('educacion', mode_per_work, 'trabajo'), axis=1)

In [27]:
mode_per_work = test.dropna(subset=['educacion']).groupby('trabajo')['educacion'].agg(lambda x: pd.Series.mode(x).iloc[0])
mode_per_work

trabajo
administración                 grado_universitario
administrador                  grado_universitario
desempleado                    grado_universitario
empleada_domestica                primaria_4_grado
empleado_ por_cuenta_propia    grado_universitario
emprendedor                    grado_universitario
estudiante                     secundaria_completa
retirado                          primaria_4_grado
servicios                      secundaria_completa
tecnico                          curso_profesional
trabajador_industria              primaria_3_grado
Name: educacion, dtype: object

In [28]:
test['educacion'] = test.apply(fill_with_mode, args=('educacion', mode_per_work, 'trabajo'), axis=1)

In [29]:
#conteo_valores_nulos(df)

#### 3.2.2.2. Completamos Trabajo con la moda de sus grupos respectivos en Educacion

In [30]:
mode_per_education = df.dropna(subset=['trabajo']).groupby('educacion')['trabajo'].agg(lambda x: pd.Series.mode(x).iloc[0])
mode_per_education

educacion
analfabeto             trabajador_industria
curso_profesional                   tecnico
grado_universitario           administrador
primaria_3_grado       trabajador_industria
primaria_4_grado       trabajador_industria
primaria_6_grado       trabajador_industria
secundaria_completa           administrador
Name: trabajo, dtype: object

In [31]:
df['trabajo'] = df.apply(fill_with_mode, args=('trabajo', mode_per_education, 'educacion'), axis=1)

In [32]:
mode_per_education = test.dropna(subset=['trabajo']).groupby('educacion')['trabajo'].agg(lambda x: pd.Series.mode(x).iloc[0])
mode_per_education

educacion
analfabeto                      emprendedor
curso_profesional                   tecnico
grado_universitario           administrador
primaria_3_grado       trabajador_industria
primaria_4_grado       trabajador_industria
primaria_6_grado       trabajador_industria
secundaria_completa           administrador
Name: trabajo, dtype: object

In [33]:
test['trabajo'] = test.apply(fill_with_mode, args=('trabajo', mode_per_education, 'educacion'), axis=1)

In [34]:
#conteo_valores_nulos(test)

### 3.2.3. Imputando data en Education y Work con sus modas globales

In [35]:
imputer = SimpleImputer(strategy='most_frequent')
df['trabajo'] = imputer.fit_transform(df[['trabajo']]).ravel()
df['educacion'] = imputer.fit_transform(df[['educacion']]).ravel()

In [36]:
imputer = SimpleImputer(strategy='most_frequent')
test['trabajo'] = imputer.fit_transform(test[['trabajo']]).ravel()
test['educacion'] = imputer.fit_transform(test[['educacion']]).ravel()

In [37]:
conteo_valores_nulos(test)

,column_name,total_nan,percentage_nan
12,dias_ultimo_contacto,12276,96.139087
0,edad,0,0.000000
1,trabajo,0,0.000000
18,ti3m,0,0.000000
17,ict,0,0.000000
16,ipc,0,0.000000
15,tasa_variacion_empleo,0,0.000000
14,resultado_camp_anterior,0,0.000000
13,contactos_prev_realizados,0,0.000000
11,num_contact_realizados,0,0.000000


## 3.3. Guardando avance en formato parquet

In [38]:
df.to_parquet('train.parquet', index=False)

In [39]:
test.to_parquet('test.parquet', index=False)

In [40]:
y.to_parquet('y.parquet', index=False)

# 4. Transformando datos

In [41]:
df = pd.read_parquet('train.parquet')
test = pd.read_parquet('test.parquet')
y = pd.read_parquet('y.parquet')

In [42]:
pd.set_option('future.no_silent_downcasting', True)

In [43]:
# Convertimos todos los "si" y "no" en 1 y 0 respectivamente.
df.replace({'si':1, 'no':0},inplace=True)
test.replace({'si':1, 'no':0},inplace=True)
y.replace({'si':1, 'no':0},inplace=True)

In [44]:
y['subscripcion_deposito'] = y['subscripcion_deposito'].astype(int)

In [45]:
df['default_credit'] = df['default_credit'].astype(int)
df['prest_vivienda'] = df['prest_vivienda'].astype(int)
df['prest_personal'] = df['prest_personal'].astype(int)

In [46]:
test['default_credit'] = test['default_credit'].astype(int)
test['prest_vivienda'] = test['prest_vivienda'].astype(int)
test['prest_personal'] = test['prest_personal'].astype(int)

In [47]:
#conteo_valores_unicos(df, 'default_credit')

In [48]:
#conteo_valores_unicos(df, 'subscripcion_deposito')

In [49]:
#df.default_credit.var()
#df.dias_ultimo_contacto.var()

In [50]:
# Calculo de la varianza de las columnas de datos
variances = df[['edad', 'default_credit', 'duracion', 'num_contact_realizados', 'dias_ultimo_contacto', 'contactos_prev_realizados', 'tasa_variacion_empleo', 'ipc', 'ict', 'ti3m', 'num_trabajadores']].var()
low_variance_cols = variances[variances < 0.01].index
print("Columnas con varianza baja:", low_variance_cols)

Columnas con varianza baja: Index(['default_credit'], dtype='object')


In [51]:
df.drop('default_credit', axis=1, inplace=True)
test.drop('default_credit', axis=1, inplace=True)

In [52]:
#df.sample(n=10)

In [53]:
#df[['subscripcion_deposito', 'dias_ultimo_contacto']].value_counts(dropna=False)

In [54]:
#conteo_valores_unicos(df, 'subscripcion_deposito')*100/len(df)

In [55]:
conteo_valores_unicos(df, 'dias_ultimo_contacto')*100/len(df)

dias_ultimo_contacto
NaN     96.367324
3.0      1.054152
6.0      0.983465
4.0      0.288893
9.0      0.156740
7.0      0.153666
12.0     0.138300
2.0      0.135227
5.0      0.110640
10.0     0.098347
13.0     0.095273
15.0     0.064540
11.0     0.064540
1.0      0.052247
14.0     0.049173
8.0      0.049173
0.0      0.036880
16.0     0.033807
17.0     0.018440
18.0     0.012293
22.0     0.009220
19.0     0.009220
21.0     0.006147
25.0     0.003073
26.0     0.003073
27.0     0.003073
20.0     0.003073
Name: count, dtype: float64

In [56]:
# Eliminaremos "dias_ultimo_contacto" porque no se puede imputar al 96% de datos.
df.drop('dias_ultimo_contacto', axis=1, inplace=True)
test.drop('dias_ultimo_contacto', axis=1, inplace=True)

In [57]:
#conteo_valores_unicos(df, 'resultado_camp_anterior')

In [58]:
#df[['resultado_camp_anterior','subscripcion_deposito']].value_counts()*100/len(df)

In [59]:
#test

In [60]:
# conteo_valores_unicos(df, 'educacion')

In [61]:
# Definir el orden específico en un diccionario
order_mapping = {
    'analfabeto': 0,
    'primaria_3_grado': 1,
    'primaria_4_grado': 2,
    'primaria_6_grado': 3,
    'secundaria_completa': 4,
    'curso_profesional': 5,
    'grado_universitario': 6
}

# Aplicar el mapeo a la columna
df['education_encoded'] = df['educacion'].map(order_mapping)

# Aplicar el mapeo a la columna
test['education_encoded'] = test['educacion'].map(order_mapping)

In [62]:
#df

In [63]:
#test

In [64]:
conteo_valores_unicos(df, 'trabajo')

trabajo
administrador                  8448
trabajador_industria           7379
tecnico                        5338
servicios                      3131
administración                 2328
retirado                       1335
emprendedor                    1125
empleado_ por_cuenta_propia    1114
empleada_domestica              838
desempleado                     817
estudiante                      685
Name: count, dtype: int64

In [65]:
#df.drop('trabajo_mapped', axis=1, inplace=True)

In [66]:
order_mapping = {
    'estudiante': 'estudiante',
    'retirado': 'jubilado',
    'desempleado': 'sin_empleo',
    'administrador': 'con_empleo',
    'tecnico': 'con_empleo',
    'trabajador_industria': 'con_empleo',
    'servicios': 'con_empleo',
    'administración': 'con_empleo',
    'empleado_ por_cuenta_propia': 'con_empleo', #auto_empleo
    'empleada_domestica': 'con_empleo',
    'emprendedor': 'con_empleo', #auto_empleo
}

df['trabajo_mapped'] = df['trabajo'].map(order_mapping)
test['trabajo_mapped'] = test['trabajo'].map(order_mapping)

In [67]:
print(conteo_valores_unicos(df, 'trabajo_mapped'))
print(conteo_valores_unicos(test, 'trabajo_mapped'))

trabajo_mapped
con_empleo    29701
jubilado       1335
sin_empleo      817
estudiante      685
Name: count, dtype: int64
trabajo_mapped
con_empleo    11638
jubilado        551
sin_empleo      308
estudiante      272
Name: count, dtype: int64


In [68]:
df = pd.get_dummies(df,prefix=["trab"],columns=["trabajo_mapped"])

In [69]:
test = pd.get_dummies(test,prefix=["trab"],columns=["trabajo_mapped"])

In [70]:
# dataset = pd.read_parquet('dataset.parquet')
df.drop(['trabajo','educacion'], axis=1, inplace=True)

In [71]:
test.drop(['trabajo','educacion'], axis=1, inplace=True)

In [72]:
df = pd.get_dummies(df,prefix=["civil","contact","mes","dia","res_camp_ant"],columns=["estado_civil", "contact", "mes", "dia_semana", "resultado_camp_anterior"])
test = pd.get_dummies(test,prefix=["civil","contact","mes","dia","res_camp_ant"],columns=["estado_civil", "contact", "mes", "dia_semana", "resultado_camp_anterior"])

In [73]:
df.to_parquet('dataset_x_train.parquet', index=False)
test.to_parquet('dataset_x_test.parquet', index=False)
y.to_parquet('dataset_y_train.parquet', index=False)

In [74]:
from sklearn.preprocessing import MinMaxScaler

In [75]:
scaler = MinMaxScaler()
df_scaled = scaler.fit_transform(df)
df_scaled = pd.DataFrame(df_scaled, columns=df.columns)

In [76]:
scaler = MinMaxScaler()
test_scaled = scaler.fit_transform(test)
test_scaled = pd.DataFrame(test_scaled, columns=test.columns)

In [77]:
df.to_parquet('dataset_x_train_minmax.parquet', index=False)
test.to_parquet('dataset_x_test_minmax.parquet', index=False)
y.to_parquet('dataset_y_train_minmax.parquet', index=False)

In [78]:
rdf = pd.read_parquet("dataset_x_train.parquet")